In [1]:
from ga_regression import *
from ga_dataset import *
from matplotlib import pyplot as plt
from functools import partial
import PIL
import matplotlib as mpl
from matplotlib.cm import ScalarMappable
from pvutils import iter_subplots
from pyvista import PolyData
import pyvista as pv
from scipy.stats import pearsonr

C:\Users\dg\.conda\envs\diffnet4\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [101]:
@dataclass
class SourceModel:
    trained_file: str
    trained_idx: int
    cp_data_file: str

    def load_corpus_data(
        self,
        model_idx: int,
    ):
        reader = Readers.from_file(self.trained_file)[self.trained_idx]
        
        ga_ndata = NeurophysData.load_data(
            data_file=reader.metadata.opts.data_file,
            file_mode=reader.metadata.opts.mesh_file_mode,
            spike_window=reader.metadata.spike_window,
            n_faces=reader.metadata.n_faces,
            features=None,
            n_min_reps=3,
        )
    
        ga_responses = ga_ndata.responses.loc[:, reader.metadata.channel]
        ga_r_min = ga_responses.min(axis=0)
        ga_r_max = ga_responses.max(axis=0)
    
        cp_ndata = NeurophysData.load_data(
            data_file=self.cp_data_file,
            file_mode='thumbnail',
            spike_window=reader.metadata.spike_window,
            n_faces=None,
            features=None,
            n_min_reps=None,
        )
    
        # Normalize to ga min/max
        cp_responses = (cp_ndata.responses.loc[:, reader.metadata.channel] - ga_r_min) / (ga_r_max - ga_r_min)
        cp_responses.replace([np.inf, -np.inf], 0, inplace=True)
    
        # Replace scene ids with filenames
        cp_responses = cp_responses.set_index(cp_ndata.scenes.filename.loc[cp_responses.index])
    
        # Update column indices
        tuples = [(model_idx, i) for i in range(len(reader.metadata.channel))]
        cp_responses.columns = pd.MultiIndex.from_tuples(tuples, names=["model_idx", "channel_idx"])
        return cp_responses.fillna(0)


In [102]:
models = [
    SourceModel(
        trained_file=r"D:\resynth\run_09_10\run00009_resynth\2025-08-08-12-28-22\opts_and_metadata.pt",
        trained_idx=5,
        cp_data_file=r"D:\resynth\run_09_10\run00010_exported\run00010_exported.hdf",
    ),
    SourceModel(
        trained_file=r"D:\resynth\run_20_21\run00020_resynth\2025-08-09-07-18-37\opts_and_metadata.pt",
        trained_idx=5,
        cp_data_file=r"D:\resynth\run_20_21\run00021_exported\run00021_exported.hdf",
    ),
    SourceModel(
        trained_file=r"D:\resynth\run_38_39\run00038_resynth\2025-08-09-17-21-56\opts_and_metadata.pt",
        trained_idx=5,
        cp_data_file=r"D:\resynth\run_38_39\run00039_exported\run00039_exported.hdf",
    ),
    SourceModel(
        trained_file=r"D:\resynth\run_42_43\run00042_resynth\2025-08-10-07-01-41\opts_and_metadata.pt",
        trained_idx=5,
        cp_data_file=r"D:\resynth\run_42_43\run00043_exported\run00043_exported.hdf",
    ),
    SourceModel(
        trained_file=r"D:\resynth\run_48_49\run00048_resynth\2025-08-11-06-09-16\opts_and_metadata.pt",
        trained_idx=5,
        cp_data_file=r"D:\resynth\run_48_49\run00049_exported\run00049_exported.hdf",
    ),
    SourceModel(
        trained_file=r"D:\resynth\run_51_52\run00051_resynth\2025-08-07-12-07-53\opts_and_metadata.pt",
        trained_idx=5,
        cp_data_file=r"D:\resynth\run_51_52\run00052_exported\run00052_exported.hdf",
    ),
]
cp_responses = pd.concat([
    tm.load_corpus_data(model_idx=i)
    for i, tm in enumerate(models)
], axis=1)

cp_responses = cp_responses.dropna()  # bah

In [103]:
mag = cp_responses.sum(axis=1)
best_stim = cp_responses.iloc[mag.argmax()]

In [104]:
for tm, (model_idx, target) in zip(models, best_stim.groupby('model_idx')):
    s = f"""
    - (): cemetery.diffnet.TrainedModel
      file: '{tm.trained_file}'
      idx: {tm.trained_idx}
      target: [{', '.join(f'{x:.04f}' for x in target.values)}]"""
    print(s[1:])

    - (): cemetery.diffnet.TrainedModel
      file: 'D:\resynth\run_09_10\run00009_resynth\2025-08-08-12-28-22\opts_and_metadata.pt'
      idx: 5
      target: [0.0556, 0.5128, 0.0000, 0.7288, 0.7627, 0.9767, 0.1111, 0.0417, 0.4667, 0.3333]
    - (): cemetery.diffnet.TrainedModel
      file: 'D:\resynth\run_20_21\run00020_resynth\2025-08-09-07-18-37\opts_and_metadata.pt'
      idx: 5
      target: [0.4638, 0.4918, 0.0000, 0.0388, 0.2182, 1.2889, 0.1852, 0.7455, 0.3400, 0.5111, 0.3448, 0.1667, 0.3714, 0.1053, 0.9259, 0.4500, 0.4167, 0.9286, 0.4545, 1.5769]
    - (): cemetery.diffnet.TrainedModel
      file: 'D:\resynth\run_38_39\run00038_resynth\2025-08-09-17-21-56\opts_and_metadata.pt'
      idx: 5
      target: [0.7292, 1.0625, 0.6087, 0.3333, 0.6389, 0.8000, 0.6250, 0.9200, 0.0476, 0.4444, 0.7778, 0.3636, 0.5000, 0.3529]
    - (): cemetery.diffnet.TrainedModel
      file: 'D:\resynth\run_42_43\run00042_resynth\2025-08-10-07-01-41\opts_and_metadata.pt'
      idx: 5
      target: [0.00

In [100]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(cp_responses);

x = pca.transform(cp_responses)
plt.scatter(x[:, 0], x[:, 1], 12, mag)

ValueError: Input X contains infinity or a value too large for dtype('float64').